#### Analyze samples saved from generated_save_samples

In [10]:
# load the results of the saved samples
from sample_functions import *

In [32]:
this_fen = '6k1/p6p/2p3p1/2K1B1P1/1P2PP1P/7r/8/8 b - - 2 43'
true_board = chess.Board(this_fen)
new_board, prob_sample_actions, cost_info = sample_construal(true_board)
print(np.prod(prob_sample_actions))

5.688888888888891e-07


In [2]:
import os
import chess
import chess.engine
import pandas as pd
import numpy as np
import json
import os 
import time


In [3]:
# order doesn't matter, so just list the result of the folder
# SET UP PARAMETERS FOR THIS JOBS
sample_params = {}
sample_params['n_samples'] = 50 # 50 samples per move?
sample_params['n_alternate_moves'] = 5
sample_params['eval_depth_consideration'] = 13
sample_params['eval_depth_evaluation'] = 13
sample_params['p_drop'] = .35
sample_params['p_move'] = 0

In [4]:
on_cluster = False

if on_cluster:
    to_save_folder = "/scratch/gpfs/erussek/Chess_Construals/sample_res/August_2019"
else:
    to_save_folder = "/Users/erussek/Dropbox/Griffiths_Lab_Stuff/Data/Chess_Construals"

param_folder_name = '_'.join([f"{key}-{value}" for key, value in sample_params.items()])
to_save_folder_params = os.path.join(to_save_folder, param_folder_name)
to_save_folder_params

'/Users/erussek/Dropbox/Griffiths_Lab_Stuff/Data/Chess_Construals/n_samples-50_n_alternate_moves-5_eval_depth_consideration-13_eval_depth_evaluation-13_p_drop-0.35_p_move-0'

In [33]:
# Dictionary to store all loaded JSON data
saved_file_names = os.listdir(to_save_folder_params)
N_Files = len(saved_file_names)

file_idx = 0
this_file_path = os.path.join(to_save_folder_params,saved_file_names[file_idx])

with open(this_file_path, 'r') as file:
    file_data = json.load(file)


# Iterate through all files in the folder
#for filename in os.listdir(to_save_folder_params):
#    if filename.endswith('.json'):
#        file_path = os.path.join(to_save_folder_params, filename)
#        with open(file_path, 'r') as file:
#            # Load JSON data and store it in the dictionary
            # Use the filename (without extension) as the key
#            json_data = json.load(file)
#            all_json_data[os.path.splitext(filename)[0]] = json_data

In [60]:

# now compute likelihood of a given cost function... 
# each index in file_data is a move

# sample parameters (we'll loop through these...)
beta_move = 500
beta_construal = 1
cost_per_piece = .05

n_moves = len(file_data)
log_lik_move = np.zeros(n_moves)

for move_idx in range(n_moves):

    sample_data = file_data[move_idx]

    illegal_move = np.array(sample_data['illegal_move'])
    U_c_i = np.array(sample_data['U_c_i']) # for each sample...
    C_c_i = cost_per_piece*np.array(sample_data['n_pieces'])

    p_i = np.exp(beta_construal*(U_c_i - C_c_i))

    q_i = sample_data['q_i'] # this is sometimes 0 - need to redo and save this value even for illegal moves.

    w_i = p_i/q_i

    # set w_i to 0 for illegal moves
    w_i[illegal_move] = 0 # change this later

    # now need to define comp_prob_move_g_construal...
    move_scores = np.array(sample_data['move_scores']) # N_samples X N_moves... # selected move is at position 0

    n_samples = move_scores.shape[0]

    sample_idx = 0

    # note that move_scores for illegal moves is 0
    sample_liks = np.array([np.exp(beta_move*move_scores[sample_idx,0])/np.sum(np.exp(beta_move*move_scores[sample_idx,:])) for sample_idx in range(n_samples)])

    log_lik_move[move_idx] = np.log(np.dot(w_i / np.sum(w_i), sample_liks))

/var/folders/s3/06m5bv8s03v5c2y_thvlc6qr0000gp/T/ipykernel_73973/1501659263.py:24: RuntimeWarning: divide by zero encountered in divide
  w_i = p_i/q_i


In [62]:
# function to compute the log likelihood of move given the parameters

def log_lik_move_fn(beta_move, beta_construal, cost_per_piece, file_data):
    n_moves = len(file_data)
    log_lik_move = np.zeros(n_moves)

    for move_idx in range(n_moves):

        sample_data = file_data[move_idx]

        illegal_move = np.array(sample_data['illegal_move'])
        U_c_i = np.array(sample_data['U_c_i']) # for each sample...
        C_c_i = cost_per_piece*np.array(sample_data['n_pieces'])

        p_i = np.exp(beta_construal*(U_c_i - C_c_i))

        q_i = sample_data['q_i'] # this is sometimes 0 - need to redo and save this value even for illegal moves.

        w_i = p_i/q_i

        # set w_i to 0 for illegal moves
        w_i[illegal_move] = 0 # change this later

        # now need to define comp_prob_move_g_construal...
        move_scores = np.array(sample_data['move_scores']) # N_samples X N_moves... # selected move is at position 0

        n_samples = move_scores.shape[0]

        sample_idx = 0

        # note that move_scores for illegal moves is 0
        sample_liks = np.array([np.exp(beta_move*move_scores[sample_idx,0])/np.sum(np.exp(beta_move*move_scores[sample_idx,:])) for sample_idx in range(n_samples)])

        log_lik_move[move_idx] = np.log(np.dot(w_i / np.sum(w_i), sample_liks))

    return np.sum(log_lik_move)

(648,)

In [52]:
illegal_move

array([False,  True, False, False, False, False,  True, False,  True,
        True, False,  True, False, False, False, False, False, False,
       False,  True,  True, False, False, False, False, False,  True,
       False, False, False, False, False, False,  True,  True, False,
        True, False, False, False, False, False,  True, False,  True,
        True,  True, False, False, False])